In [10]:
import arcpy
import os
from arcpy import env
import pandas as pd
import string

arcpy.env.workspace = gdb = r"C:\Users\vpavao\OneDrive - University of Massachusetts\Documents\ArcGIS\Projects\Wells_Sandbox_Oct\New_Workspace.gdb"
arcpy.env.overwriteOutput = True

### Examine parcel data
parcels_east = r"C:\Users\vpavao\OneDrive - University of Massachusetts\Documents\ArcGIS\Projects\Wells_Sandbox_Oct\data\L3_Data\L3_TAXPAR_POLY_ASSESS_EAST.shp"
parcels_west = r"C:\Users\vpavao\OneDrive - University of Massachusetts\Documents\ArcGIS\Projects\Wells_Sandbox_Oct\data\L3_Data\L3_TAXPAR_POLY_ASSESS_WEST.shp"
structs = r"C:\Users\vpavao\OneDrive - University of Massachusetts\Documents\ArcGIS\Projects\Wells_Sandbox_Oct\data\structures\STRUCTURES_POLY.shp"
pws_service_data = r"C:\Users\vpavao\OneDrive - University of Massachusetts\Documents\ArcGIS\Projects\Wells_Sandbox_Oct\data\PWS_boundaries\PWS_WATER_SERVICE_AREA_COMM_POLY.shp"
pws_source_data = r"C:\Users\vpavao\OneDrive - University of Massachusetts\Documents\ArcGIS\Projects\Wells_Sandbox_Oct\data\PWS\PWSDEP_PT.shp"
wlv_points = r"C:\Users\vpavao\OneDrive - University of Massachusetts\Documents\ArcGIS\Projects\Wells_Sandbox_Oct\data\WLV_2023\WLV_2023.shp"
townsshp = r"C:\Users\vpavao\OneDrive - University of Massachusetts\Documents\ArcGIS\Projects\Wells_Sandbox_Oct\data\Towns\TOWNSSURVEY_POLY.shp"
major_basins_data = r"C:\Users\vpavao\OneDrive - University of Massachusetts\Documents\ArcGIS\Projects\Wells_Sandbox_Oct\data\Major_Basins\MAJBAS_POLY.shp"
subbasins = r"C:\Users\vpavao\OneDrive - University of Massachusetts\Documents\ArcGIS\Projects\Wells_Sandbox_Oct\data\Subbasins\SUBBASINS_POLY.shp"

arcpy.management.AddSpatialIndex(parcels_east)
arcpy.management.AddSpatialIndex(parcels_west)
arcpy.management.AddSpatialIndex(structs)
    
# Code below can be used to examine field information for parcels data
#fields = arcpy.ListFields(east_parc)
#for field in fields:
#    print(f"{field.name} has a type of {field.type} with a length of {field.length}")

#Import excel queries
res_file = pd.ExcelFile(r"C:\Users\vpavao\OneDrive - University of Massachusetts\Documents\Task3\Appendix N-queries\residential_use_codes.xlsx")
unc_file = pd.ExcelFile(r"C:\Users\vpavao\OneDrive - University of Massachusetts\Documents\Task3\Appendix N-queries\uncertain_use_codes.xlsx")
dev_land_file = pd.ExcelFile(r"C:\Users\vpavao\OneDrive - University of Massachusetts\Documents\Task3\Appendix N-queries\developable_land_use_codes.xlsx")
bldg_style_file = pd.ExcelFile(r"C:\Users\vpavao\OneDrive - University of Massachusetts\Documents\Task3\Appendix N-queries\residential_styles.xlsx")
city_query_file = pd.ExcelFile(r"C:\Users\vpavao\OneDrive - University of Massachusetts\Documents\Task3\Appendix N-queries\city_query.xlsx")

res_df = res_file.parse('resi use code query')
unc_df = unc_file.parse('uncertain use codes')
dev_df = dev_land_file.parse('developable_land')
bldg_df = bldg_style_file.parse('GREEN STYLES')
city_df = city_query_file.parse('Sheet1')

res_codes_raw = res_df["Residential use codes"].tolist()
unc_codes_raw = unc_df["Uncertain Use Codes"].tolist()
dev_codes_raw = dev_df['Residential "developable land" use codes'].tolist()
bldg_codes_raw = bldg_df["Residential styles to query in Step 7 of the methodology"].tolist()
city_query = city_df["TOWN"].tolist()
#print(city_query)

res_codes = []
for num in res_codes_raw:
    ###  Assume all codes from table are strings
    ### Remove doubly-wrapped quotes
    c = num.replace('"', '')
    cd = c.replace('\'', '')
    res_codes.append(cd)

unc_codes = []
for num in unc_codes_raw:
    c = num.replace('"', '')
    cd = c.replace('\'', '')
    unc_codes.append(cd)

dev_codes = []
for num in dev_codes_raw:
    c = num.replace('"', '')
    cd = c.replace('\'', '')
    dev_codes.append(cd)

#Selection function for queries
def select_in(fc, field, values, out_fc):
    # ensure all values are quoted as strings
    vals = [str(v).replace("'", "''") for v in values]
    chunks = []
    # handle 1000-term limits safely (esp. file gdb is fine, some datasources limit)
    for i in range(0, len(vals), 900):
        part = f"{arcpy.AddFieldDelimiters(fc, field)} IN ('" + "','".join(vals[i:i+900]) + "')"
        chunks.append(part)
    where = " OR ".join(chunks)
    arcpy.analysis.Select(fc, out_fc, where)

### 3. Export parcels with residential use codes to separate files
res_out_east = "res_out_east"
res_out_west = "res_out_west"
select_in(parcels_east, "USE_CODE", res_codes, res_out_east)
select_in(parcels_west, "USE_CODE", res_codes, res_out_west)

### 4. Export parcels with uncertain use codes to separate files
unc_out_east = "unc_out_east"
unc_out_west = "unc_out_west"
unc_filter_out_east = "uncfilte"
unc_filter_out_west = "uncfiltw"
    

def generate_uncertain_layers():
    select_in(parcels_east, "USE_CODE", unc_codes, unc_out_east)
    select_in(parcels_west, "USE_CODE", unc_codes, unc_out_west)
    arcpy.analysis.Select(unc_out_east, unc_filter_out_east, "BLD_AREA - RES_AREA > 0")
    arcpy.analysis.Select(unc_out_west, unc_filter_out_west, "BLD_AREA - RES_AREA > 0")
    arcpy.management.Delete([unc_out_east, unc_out_west])
    
generate_uncertain_layers()

###Developable use codes w/structures over 50 sq meters
dev_out_east = "dev_out_east"
dev_out_west = "dev_out_west"
dev_filter_out_east = "dev_struct_out_east"
dev_filter_out_west = "dev_struct_out_west"

dev_structs = "structs_over_50sqm"

def generate_dev_layers():
    # Filter parcels for developable land use codes
    select_in(parcels_east, "USE_CODE", dev_codes, dev_out_east)
    select_in(parcels_west, "USE_CODE", dev_codes, dev_out_west)

    # Filter structures by area
    arcpy.analysis.Select(structs, dev_structs, "SHAPE_AREA > 50")

    #Select parcels with filtered structures
    arcpy.analysis.PairwiseIntersect([dev_out_east, dev_structs], dev_filter_out_east)
    arcpy.analysis.PairwiseIntersect([dev_out_west, dev_structs], dev_filter_out_west)

    arcpy.management.Delete([dev_out_east, dev_out_west])
    
generate_dev_layers()

### Residential style codes
bldg_style_east = "bldg_style_east"
bldg_style_west = "bldg_style_west"
select_in(parcels_east, "STYLE", bldg_codes_raw, bldg_style_east)
select_in(parcels_west, "STYLE", bldg_codes_raw, bldg_style_west)

### Merge parcels together and delete duplicates
ma_resi_parcels = "ma_resi_parcels"
def merge_east_west():
    arcpy.management.Merge([res_out_east, res_out_west], "ma_res")
    arcpy.management.Merge([unc_filter_out_east, unc_filter_out_west], "ma_unc")
    arcpy.management.Merge([dev_filter_out_east, dev_filter_out_west], "ma_dev")
    arcpy.management.Merge([bldg_style_east, bldg_style_west], "ma_style")
    arcpy.management.Merge(["ma_dev", "ma_unc", "ma_res", "ma_style"], ma_resi_parcels)
    arcpy.management.DeleteIdentical(ma_resi_parcels, ["LOC_ID"])

merge_east_west()
arcpy.management.AddSpatialIndex(ma_resi_parcels)

### Misc. queries for filtering study parcels. 
# These four queries were developed during the final QA/QC of the MA Resi Parcels layer. The
# queried parcel attributes were non-residential based on sampling parcels throughout the state.

output_layer_17 = "filtparc_17a"
output_layer_18 = "filtparc_18"
output_layer_19 = "filtparc_19"
output_layer_20 = "Resi_Parcels"

def filter_misc_parcels():
    ###Style = null; Bldg_Area = 0; Res_Area = 0; Address number = 0; and a structure from the 2019 MassGIS 2D Structures layer is not in the parcel.
    
    ### First select by attributes, and then select by location on that resulting layer
    arcpy.management.SelectLayerByLocation(parcels_outside_pws, "INTERSECT", structs,"","NEW_SELECTION")
    arcpy.management.SelectLayerByAttribute(parcels_outside_pws, "SWITCH_SELECTION")
    
    where_clause_1 = "STYLE = '' AND BLD_AREA = 0 AND RES_AREA = 0 AND ADDR_NUM = '0'"
    arcpy.management.SelectLayerByAttribute(parcels_outside_pws, "SUBSET_SELECTION", where_clause_1)
    arcpy.management.SelectLayerByAttribute(parcels_outside_pws, "SWITCH_SELECTION")
    arcpy.management.CopyFeatures(parcels_outside_pws, output_layer_17)
    arcpy.management.SelectLayerByAttribute(parcels_outside_pws, "CLEAR_SELECTION")

    ###17b. Use_Code = 013, 014, 016, 0160, 017, 0170, 018, 0180, 959, 9590, 1333, or 071; Year_Built < 2015; and a structure from the 2019 MassGIS 2D Structures layer is not in the parcel.
    arcpy.management.SelectLayerByLocation(output_layer_17, "INTERSECT", structs,"","NEW_SELECTION")
    arcpy.management.SelectLayerByAttribute(output_layer_17, "SWITCH_SELECTION")

    filterCodes = ['013', '014', '016', '0160', '017', '0170', '018', '0180', '959', '9590', '1333', '071']
    for code in filterCodes:
        print(code)
        where_clause_2 = "USE_CODE = '{}'".format(code)
        arcpy.management.SelectLayerByAttribute(output_layer_17, "SUBSET_SELECTION", where_clause_2)
    arcpy.management.SelectLayerByAttribute(output_layer_17, "SUBSET_SELECTION", "YEAR_BUILT < 2015")
    ### TODO Ask if YEAR_BUILT < 2015 selecting all parcels with 0 is intentional, should it follow a four-digit number?
    arcpy.management.SelectLayerByAttribute(output_layer_17, "SWITCH_SELECTION")
    arcpy.management.CopyFeatures(output_layer_17, output_layer_18)
    arcpy.management.SelectLayerByAttribute(output_layer_17, "CLEAR_SELECTION")

    ###17c. Address number = 0; Res_Area = 0; and use_code does NOT = 101, 1010, 102, 1020, 103, 1040, 109, 1090, 130, 1300, 131, or 1310. 
    filterCodes2 = ['101', '1010', '102', '1020', '103', '1040', '109', '1090', '130', '1300', '131', '1310']
    for code in filterCodes2:
        print(code)
        where_clause_3 = "USE_CODE = '{}'".format(code)
        arcpy.management.SelectLayerByAttribute(output_layer_18, "ADD_TO_SELECTION", where_clause_3)
    arcpy.management.SelectLayerByAttribute(output_layer_18, "SWITCH_SELECTION")

    where_clause_4 = "RES_AREA = 0 AND ADDR_NUM = '0'"
    arcpy.management.SelectLayerByAttribute(output_layer_18, "SUBSET_SELECTION", where_clause_4)

    arcpy.management.SelectLayerByAttribute(output_layer_18, "SWITCH_SELECTION")
    arcpy.management.CopyFeatures(output_layer_18, output_layer_19)
    arcpy.management.SelectLayerByAttribute(output_layer_18, "CLEAR_SELECTION")
    
    ### 17d. Style = 'outbuildings'; and City does NOT = Rehoboth 
    where_clause_5 = "STYLE = 'Outbuildings' And NOT CITY = 'REHOBOTH'"
    arcpy.management.SelectLayerByAttribute(output_layer_19, "NEW_SELECTION", where_clause_5)
    arcpy.management.SelectLayerByAttribute(output_layer_19, "SWITCH_SELECTION")
    arcpy.management.CopyFeatures(output_layer_19, output_layer_20)
    arcpy.management.SelectLayerByAttribute(output_layer_19, "CLEAR_SELECTION")

filter_misc_parcels()

### Subtract parcels inside PWS service areas

def query_from_pws(parc_layer, service_layer, points_lyr, pws_parc_output_name, nonpws_parc_output_name):
    # Select only active PWS service areas 
    active_pws = "active_pws"
    active_pts = "active_pts"
    arcpy.analysis.Select(service_layer, active_pws, "\"PWS_STATUS\" = 'A'")
    arcpy.management.AddSpatialIndex(active_pws)

    arcpy.analysis.Select(points_lyr, active_pts, "\"TYPE\" <> 'ESW' OR \"TYPE\" <> 'PW'")
    arcpy.management.AddSpatialIndex(active_pts)
                                            
    arcpy.management.SelectLayerByLocation(parc_layer, "INTERSECT", active_pws,"","NEW_SELECTION")
    arcpy.management.SelectLayerByLocation(parc_layer, "INTERSECT", active_pts,"","ADD_TO_SELECTION")
    arcpy.management.CopyFeatures(parc_layer, pws_parc_output_name)
    arcpy.management.SelectLayerByAttribute(parc_layer, "SWITCH_SELECTION")
    arcpy.management.CopyFeatures(parc_layer, nonpws_parc_output_name)
    arcpy.management.SelectLayerByAttribute(parc_layer, "CLEAR_SELECTION")

pws_service_areas = "PWS_Service_Areas"
pws_source_points = "PWS_Source_Points"
parcels_pws = "PWS_Parc"
parcels_outside_pws = "Non_PWS_Parc"
arcpy.management.MakeFeatureLayer(pws_service_data, pws_service_areas)
arcpy.management.MakeFeatureLayer(pws_source_data, pws_source_points)

query_from_pws(ma_resi_parcels, pws_service_areas, pws_source_points, parcels_pws, parcels_outside_pws)

#### 15. Subtract all parcels in city query list - this step should not be performed until further notice
'''
resi_parcels_minus_cities_list = "resi_parcels_cityfilter"
filter_study_parcels(parcels_outside_pws_output, city_query, "CITY", "Resi-Parcels-In-CitiesList", True, resi_parcels_minus_cities_list)
arcpy.management.Delete("Resi-Parcels-In-CitiesList")
'''

wlv_points_domestic = "WLV_Points_2023_Domestic"
wlv_parcels = "WLV_Confirmed_Well_Parcels"
output_layer_vfw = "Resi_Parcels_and_wellparc"

### Add verified wells from 2023 WLV point data to parcels
def add_verified_wells():
    arcpy.analysis.Select(wlv_points, wlv_points_lyr_domestic,
                                            "\"Well_Type\" = 'Domestic' AND \"Work_Perfo\" <> 'Decommission'")

    arcpy.analysis.PairwiseIntersect([output_layer_20, wlv_points_lyr_domestic], wlv_parcels)

    #arcpy.management.Delete([wlv_points_lyr])
    arcpy.management.Merge([output_layer_20, wlv_parcels], output_layer_vfw)
    arcpy.management.DeleteIdentical(output_layer_vfw, ["LOC_ID"])

add_verified_wells()

ineligible_fc = "ma_nonresiparcels"

def get_ineligible_wellparc():
    inelig_pws = "IneligiblePWS_C1"
    inelig_nopws = "IneligibleNoPWS_C2"

    arcpy.management.Merge([res_working_output_east, res_working_output_west,
                            unc_working_output_east, unc_working_output_west,
                            dev_code_working_output_east, dev_code_working_output_west,
                            final_parcels_east, final_parcels_west], ineligible_fc)

    eligible_ids = []
    with arcpy.da.SearchCursor("maresiparcels", "LOC_ID") as cursor:
        for row in cursor:
            firstval = row[0]
            print(firstval)
            eligible_ids.append(row[0])
    
    with arcpy.da.UpdateCursor(ineligible_fc, "LOC_ID") as cursor:
        for row in cursor:
            if row[0] in eligible_ids:
                cursor.deleteRow()
    '''
    arcpy.management.DeleteIdentical(ineligible_fc, ["LOC_ID"])
    '''

    query_from_pws(ineligible_fc, pws_service_areas, pws_source_points, "pws_temp", "nopws_temp")

    arcpy.analysis.PairwiseIntersect(["nopws_temp", wlv_points_lyr_domestic], inelig_nopws)
    arcpy.analysis.PairwiseIntersect(["pws_temp", wlv_points_lyr_domestic], inelig_pws)
    #arcpy.management.Delete([temp_copy, ineligible_fc])
    
#get_ineligible_wellparc()

# Delete temporary layers for each step of 17 - comment out if you need to examine the output for each
#arcpy.management.Delete([output_layer_17a, output_layer_17b, output_layer_17c])

major_basins_data, subbasins
### 18. Dissolve the parcel layer by Major Basin, subbasin, and town and create a new layer for each of
#these three scales. Sum the “water-use” parcel values per dissolved unit in each layer. Name
#the four layers as follows: MA Resi Private Wells-parcels; MA Resi Private Wells-major basins;
#MA Resi Private Wells-subbasins; and MA Resi Private Wells-towns.
def dissolve_parcel_layers():
    major_basins_lyr = "Major_Basins"
    subbasins_lyr = "Subbasins"
    out_parcels = arcpy.CreateUniqueName("MA_Resi_Private_Wells_parcels", gdb)
    out_major   = arcpy.CreateUniqueName("MA_Resi_Private_Wells_majbasins", gdb)
    out_sub     = arcpy.CreateUniqueName("MA_Resi_Private_Wells_subbasins", gdb)
    out_town    = arcpy.CreateUniqueName("MA_Resi_Private_Wells_towns", gdb)

    arcpy.management.AddSpatialIndex(ma_resi_parcels, ["LOC_ID"])
    arcpy.management.AddSpatialIndex(major_basins_lyr, ["NAME"])
    arcpy.management.AddSpatialIndex(subbasins_lyr, ["SUB_NAME"])
    
    tab_maj = arcpy.CreateUniqueName("ti_parc_majbasin", gdb)
    
    arcpy.analysis.TabulateIntersection(
        in_zone_features = ma_resi_parcels,
        zone_fields = "LOC_ID",
        in_class_features = major_basins_lyr,
        out_table = tab_maj,
        class_fields = "NAME",
        out_units = "SQUARE_METERS"
    )
    
    tab_sub = arcpy.CreateUniqueName("ti_parc_subbasin", gdb)
    
    arcpy.analysis.TabulateIntersection(
        in_zone_features = ma_resi_parcels,
        zone_fields = "LOC_ID",
        in_class_features = subbasins_lyr,
        out_table = tab_sub,
        class_fields = "SUB_NAME",
        out_units = "SQUARE_METERS"
    )
    

    # Build winners for majors
    best_major = {}  # LOC_ID -> (pct, major_name)
    with arcpy.da.SearchCursor(tab_maj, ["LOC_ID", "NAME", "PERCENTAGE"]) as rows:
        for pid, mname, pct in rows:
            if pid is None or mname is None: 
                continue
            if (pid not in best_major) or (pct > best_major[pid][0]):
                best_major[pid] = (pct, mname)

    winners_major = arcpy.CreateUniqueName("winners_major", gdb)
    arcpy.management.CreateTable(gdb, winners_major.split("\\")[-1])
    arcpy.management.AddField(winners_major, pid_field,       loc_id_type, field_length=128 if loc_id_type=="TEXT" else None)
    arcpy.management.AddField(winners_major, "AssignedMajor", "TEXT",      field_length=128)
    with arcpy.da.InsertCursor(winners_major, [pid_field, "AssignedMajor"]) as icur:
        for pid, (pct, mname) in best_major.items():
            if pct is not None and pct >= 50.0:
                icur.insertRow([pid, mname])

    # Build winners for subbasins
    best_sub = {}  # LOC_ID -> (pct, sub_name)
    with arcpy.da.SearchCursor(tab_sub, ["LOC_ID", "SUB_NAME", "PERCENTAGE"]) as rows:
        for pid, sname, pct in rows:
            if pid is None or sname is None:
                continue
            if (pid not in best_sub) or (pct > best_sub[pid][0]):
                best_sub[pid] = (pct, sname)

    winners_sub = arcpy.CreateUniqueName("winners_sub", gdb)
    arcpy.management.CreateTable(gdb, winners_sub.split("\\")[-1])
    arcpy.management.AddField(winners_sub, pid_field,     loc_id_type, field_length=128 if loc_id_type=="TEXT" else None)
    arcpy.management.AddField(winners_sub, "AssignedSub", "TEXT",      field_length=128)
    with arcpy.da.InsertCursor(winners_sub, [pid_field, "AssignedSub"]) as icur:
        for pid, (pct, sname) in best_sub.items():
            if pct is not None and pct >= 50.0:
                icur.insertRow([pid, sname])
    
    parcels_with_assign = arcpy.CreateUniqueName("parcels_with_assign", gdb)
    arcpy.management.CopyFeatures(ma_resi_parcels, parcels_with_assign)
    arcpy.management.AddField(parcels_with_assign, "AssignedMajor", "TEXT", field_length=128)
    arcpy.management.AddField(parcels_with_assign, "AssignedSub", "TEXT", field_length=128)

    arcpy.management.JoinField(parcels_with_assign, pid_field, winners_major, pid_field, ["AssignedMajor"])
    arcpy.management.JoinField(parcels_with_assign, pid_field, winners_sub,   pid_field, ["AssignedSub"])

    # Output 1: just copy to GDB
    arcpy.management.CopyFeatures(parcels_with_assign, out_parcels)

    lyr_major = arcpy.management.MakeFeatureLayer(parcels_with_assign, "parcels_major_sel", "AssignedMajor IS NOT NULL")
    arcpy.management.Dissolve(
        in_features       = lyr_major,
        out_feature_class = out_major,
        dissolve_field    = ["AssignedMajor"],
        statistics_fields = [[water_use_field, "SUM"]],
        multi_part        = "MULTI_PART"
    )

    # Dissolve by Subbasin - 50% water use required
    lyr_sub = arcpy.management.MakeFeatureLayer(parcels_with_assign, "parcels_sub_sel", "AssignedSub IS NOT NULL")
    arcpy.management.Dissolve(
        in_features       = lyr_sub,
        out_feature_class = out_sub,
        dissolve_field    = ["AssignedSub"],
        statistics_fields = [[water_use_field, "SUM"]],
        multi_part        = "MULTI_PART"
    )

    # Dissolve by Town by CITY field
    arcpy.management.Dissolve(
        in_features       = parcels_with_assign,
        out_feature_class = out_town,
        dissolve_field    = [town_name_field],
        statistics_fields = [[water_use_field, "SUM"]],
        multi_part        = "MULTI_PART"
    )

    print("Created:")
    print("Parcels:", out_parcels)
    print("Major basins (50% rule):", out_major)
    print("Subbasins (50% rule):", out_sub)
    print("Towns:", out_town)
    
dissolve_parcel_layers()

###Rename "_lyr" to fc when appropriate
###Effectively add indexes - before filters, joins, etc and after creation

013
014
016
0160
017
0170
018
0180
959
9590
1333
071
101
1010
102
1020
103
1040
109
1090
130
1300
131
1310


<class 'arcgisscripting.ExecuteError'>: Failed to execute. Parameters are not valid.
ERROR 002948: The shape type of C:\Users\vpavao\OneDrive - University of Massachusetts\Documents\ArcGIS\Projects\Wells_Sandbox_Oct\New_Workspace.gdb\WLV_Confirmed_Well_Parcels is not the same as that of previously entered datasets.
Failed to execute (Merge).
